<a href="https://colab.research.google.com/github/SummerLife/EmbeddedSystem/blob/master/MachineLearning/project/08_code_nn_from_scratch/coding_nn_with_bp_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coding a Neural Network with Backpropagation from scratch

- Initialize Network
- Forward Propagate
- Back Propagate Error
- Train Network
- Predict
- Seeds Dataset Case Study

## 1. Initialize Network

In [1]:
from random import seed
from random import random

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{"weights": [random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{"weights": [random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

seed(1)
network = initialize_network(2,1,2)
for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]


## 2. Forward Propagate

We can break forward propagation down into three parts:

1. Neuron Activation
2. Neuron Transfer
3. Forward Propagation

### 2.1 Neuron Activation

`activation = sum(weight_i * input_i) + bias`

In [2]:
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation

### 2.2 Neuron Transfer

Once a neuron is activated, we need to transfer the activation to see what the neuron output actually is.

The sigmoid activation function looks like an S shape, it's also called the logistic function. It can take any input value and produce an number between 0 and 1 on an S-curve. It is also a function of which we can easily calculate the derivatice(slope) that we will need later when backpropagating error.

We can transfer an activation function using the sigmoed function as follows:

`output = 1 / (1 + e^(-activation))`

In [3]:
# Transfer neuron activation
from math import exp

def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

### 2.3 Forward Propagation

Forward propagating an input is straightforward.

We work through each layer of our network calculating the outputs for each neuron. All of the outputs from on layer become in puts to the neurons on the next layer.

In [4]:
# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

### 2.4 Test out the forward propagation of out network



In [5]:
row = [1, 0, None]
output = forward_propagate(network, row)
print(output)

[0.6629970129852887, 0.7253160725279748]


## 3. Back Propagate Error

Error is calculated between the expected outputs and the outputs forward propagated from the network. These errors are then propagated backward through the netwoek from the output layer to the hidden layer, assigning blame for the error and updating weights as they go.

1. Transfer Derivative
2. Error Backpropagation

### 3.1 Transfer Derivative

We are using the sigmoid transfer function, the derivative of which can be calculated as follows:

```
derivative = output * (1.0 - output)
```

Using function named `transfer_derivative()` to implements this equation.

In [7]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

### 3.2 Error Backpropagation

The error for a given neuron can be calculated as follows:

`error = (expected - output) * transfer_derivative(output)`


The error signal for a neuron in the hidden layer is calculated as the weighted error of each neuron in the output layer. Think of the error traveling back along the weights of the output layer to the neurons in the hidden layer.

The back-propagated error signal is accumulated and then used to determine the error for the neuron in the hidden layer, as follows:

`error = (weight_k * error_j) * transfer_derivative(output)`

Where error_j is the error signal from the jth neuron in the output layer, weight_k is the weight that connects the kth neuron to the current neuron and output is the output for the current neuron.


In [8]:
# Backpropagate error and store in neurons

def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()

        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])

        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [9]:
# test backpropagation of error
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
		[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]

backward_propagate_error(network, expected)

for layer in network:
	print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.007668854370284511}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]
